In [ ]:
# 추가 해야할 것 새로운 데이터 생기면 그위에 업데이트 하는 거 짜야함

In [ ]:
# !pwd

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os 
from restaurant.models import *

# 첫번째 selenium 해서 데이터 가져오기

In [2]:
chrom_path= '/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/chromedriver'
rootPath = 'https://www.mangoplate.com/'

options = webdriver.ChromeOptions() 
options.add_argument('headless')
options.add_argument('disable-gpu')
driver = webdriver.Chrome(chrom_path,chrome_options=options)

driver.get(rootPath)
html = driver.page_source
r = open( '/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/home_first_page.txt', mode='w', encoding='utf-8')
r.write(html)
print(rootPath)
r.close()

driver.quit()

/home/kimdooh/.pyenv/versions/mangoplate-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


https://www.mangoplate.com/


# first page에서 각 카테고리 별 href 가져오기  


In [3]:
html = open('../home_first_page.txt', "r").read()
song_soup = BeautifulSoup(html)
soup = BeautifulSoup(html, 'html.parser')

home_div_top_list_slide= soup.select('body > main > article > section:nth-child(3) > div.slider-container.toplist-slider > div > div > div')
big_category = soup.select_one('div.module_title_wrap').select_one('h2').get_text(strip=True).replace(" ","")
home_trust_list = soup.select_one('div.slick-track')
one_ul_two_a=home_trust_list.select('ul')
category_data = {
    
}
for text in one_ul_two_a:
    select_list = text.select('a')
    select_img  = text.select('img')
    for data, img in zip(select_list, select_img) :
        category_data[data.select('span.title')[0].get_text(strip=True)] ={
            "href" : data['href'],
            "title" : data.select('span.title')[0].get_text(strip=True),
            "sub_title" :data.select('p')[0].get_text(strip=True),
            "img_uri" : img['src'],
            "big_category" : big_category,
        }
#         print("home_list big_category >>> ", big_category)
#         print("home_list href         >>>  ", data['href'])
#         print("home_list title        >>> ", data.select('span.title')[0].get_text(strip=True))
#         print("home_list distr        >>> ", data.select('p')[0].get_text(strip=True))
#         print(img['src'])
#         print('\n')

In [4]:
try:
    os.mkdir('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'+ big_category + '/')
except FileExistsError:
    print('파일 이미 있음')

파일 이미 있음


# 여기까지 돌리고!!!!!!!!!!!!!!!!!!!!!!!!!!!1

# 큰 카테고리에서 작은 카테고리별 (리스트 목록 페이지) 


# 두번째 selenium이용해서 각 가테고리별로 개별 식당 href 가져오기

In [ ]:
try:
    os.mkdir('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'\
             + big_category + '/'+'list_page/')
except FileExistsError:
    print('파일 이미 있음')
for data in category_data:
    chrom_path= '/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/chromedriver'
    rootPath = 'https://www.mangoplate.com'
    URL = rootPath + category_data[data]['href']
    
    options = webdriver.ChromeOptions() 
    options.add_argument('headless')
    options.add_argument('disable-gpu')
    driver = webdriver.Chrome(chrom_path,chrome_options=options)
    driver.get(URL)
    print(URL)
#     driver.get_screenshot_as_file(f'./data/naver_main{data}.png')
    #########################
#     더보기 버튼 눌러서 데이터 더 가져오기
    #########################
    title = category_data[data]['title']

    r = open('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'\
             +big_category+'/'+ 'list_page/'+ title.replace(" ","") + '.txt', mode='w', encoding='utf-8')
    category_data[data]['page_source'] = driver.page_source
    r.write(category_data[data]['page_source'])
    r.close
    driver.quit()

# 큰 카테고리에서 작은 카테고리별 (리스트 목록 페이지) 
# html 저장 하고 가각 리스트 가져오기

In [27]:
import os 
path='../data/'+ big_category+'/list_page/' 
file_list = [i for i in os.listdir(path) if '.txt' in i]
print(file_list)

['강릉맛집베스트30곳.txt', '부산대맛집베스트10곳.txt', '강원도원주맛집베스트20곳.txt', '피자맛집베스트30곳.txt', '연세대맛집베스트35곳.txt', '성균관대맛집베스트30곳.txt', '혼밥맛집베스트45곳.txt', '와플맛집베스트25곳.txt', '된장찌개맛집베스트7곳.txt', '망원동카페베스트20곳.txt', '서울대맛집베스트30곳.txt', '이태원브런치맛집베스트15곳.txt']


In [28]:
second_category_data_list = {
    
}
for second_category_detail_list_source in file_list:
    print(second_category_detail_list_source)
    
    html = open('../data/'+ big_category+'/list_page/' +second_category_detail_list_source, "r").read()
#     print(html)
    second_category_soup = BeautifulSoup(html)
    second_category_list = second_category_soup.select_one('ul.list-restaurants').select('li.toplist_list')
    second_category_data_href_list= {}
    for data in second_category_list:
        div_with_reivew = data.select_one('div.with-review')
        div_info = div_with_reivew.select_one('div.info')
        second_category_list_simple_data = div_with_reivew.select_one('a')

        second_category_data_href_list[div_info.select_one('h3').get_text(strip=True)] ={
            "href" : second_category_list_simple_data['href'],
            "point" : div_info.select_one('strong.point').get_text(strip=True),
            "address" : div_info.select_one('p.etc').get_text(strip=True),
            "title" :div_info.select_one('h3').get_text(strip=True),
#             "review_sample" : data.select_one('div.review-content').select_one('p.long_review').get_text(strip=True),
            "img_uri" : data.select_one('div.thumb').select_one('img')['data-original'],
        }
    second_category_data_list[second_category_detail_list_source.split('.')[0]] = second_category_data_href_list
    

강릉맛집베스트30곳.txt
부산대맛집베스트10곳.txt
강원도원주맛집베스트20곳.txt
피자맛집베스트30곳.txt
연세대맛집베스트35곳.txt
성균관대맛집베스트30곳.txt
혼밥맛집베스트45곳.txt
와플맛집베스트25곳.txt
된장찌개맛집베스트7곳.txt
망원동카페베스트20곳.txt
서울대맛집베스트30곳.txt
이태원브런치맛집베스트15곳.txt


# sub_카테고리별 파일 만들기

In [ ]:
for title in second_category_data_list.keys():
    print(title.replace(" ",""))
    os.mkdir('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'+ big_category +'/'+ title.replace(" ","") + '/')

#     try:
#         os.mkdir('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'+ big_category +'/'+ title.replace(" ","") + '/')
#         print('만듬')
#     except FileExistsError:
#         print('파일 이미 있음')
#         continue

In [ ]:
second_category_data_list

# 세번째 selenium 각각 가게별 상세 데이터 가져오기 

In [ ]:
for file_name, res_name in second_category_data_list.items():
    chrom_path= '/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/chromedriver'
    rootPath = 'https://www.mangoplate.com'
    
    for restaurant, factors in second_category_data_list[file_name].items():
        
        URL = rootPath + factors['href']
        print(URL)

        options = webdriver.ChromeOptions() 
        options.add_argument('headless')
        options.add_argument('disable-gpu')
        driver = webdriver.Chrome(chrom_path,chrome_options=options)
        driver.get(URL)
        driver.get_screenshot_as_file(f'naver_main{data}.png')

        
#         second_category_data_list[file_name][restaurant]['page_source'] = driver.page_source
        
        r = open('/home/kimdooh/projects/fastcampus/project_with_team/personal-projects/web_craw_mango/data/'+ big_category + '/'+\
        file_name.replace(" ","") +'/'+ restaurant.replace(" ","") + '.txt', mode='w', encoding='utf-8')
#         r.write(second_category_data_list[file_name][restaurant]['page_source'])
        r.write(driver.page_source)

        driver.quit()

# 여기서 부터 모든 데이터 가져다 써!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# 각 각의 가게 상세 데이터 저장한 파일로 부터 factor뽑기

In [5]:
import os 
path='../data/'+ big_category+'/' 
file_list = [i for i in os.listdir(path) if '곳' in i]

In [6]:
detail_page_craw = {}
path='../data/'+ big_category+'/' 
for category_detail_list in file_list:
    file_name=os.listdir(path+category_detail_list+ '/')
    parent_text={}
    for restaurant in file_name:
#         
#         print(detail_page_craw)
#         print(restaurant)
        
        try:
            html = open(path + category_detail_list +'/'+restaurant, "r").read()
            health_restaurant_detail = BeautifulSoup(html)
            # text 크롤링
            div_inner = health_restaurant_detail.select_one('div.column-contents').select_one('div.inner')
            section_restaurant_detail = div_inner.select_one('section.restaurant-detail')
            div_restaurant_title_wrap=section_restaurant_detail.select_one('div.restaurant_title_wrap')
            span_title = div_restaurant_title_wrap.select_one('span.title')
            restaurant_title=span_title.select_one('h1').get_text(strip=True)
            restaurant_point=span_title.select_one('strong.rate-point').select_one('span').get_text(strip=True)
            tbody = section_restaurant_detail.select_one('table.info').select_one('tbody')
            # images 크롤링
            asid_restaurant = health_restaurant_detail.select_one('aside.restaurant-photos')
            div_owl_wrapper = asid_restaurant.select_one('div.owl-wrapper')
            div_owl_item = div_owl_wrapper.select('div.owl-item')
        except AttributeError as ex:
            print(f'page 없는 데이터 존재', ex)
        ###################################################################################
        image_contents = {

        }
        for index, data in enumerate(div_owl_item):
            image_contents['image_'+str(index)] = data.select_one('meta')['content']
        restaurant_detail_fields = {
            "title" : restaurant_title,
            "images" : image_contents,
        }
    #     print(restaurant_detail_fields)
        for i in tbody.select('tr'):

            restaurant_detail_fields[i.select_one('th').get_text(strip=True)]= i.select_one('td').get_text(strip=True)


            if not i.select_one('span.Restaurant__InfoAddress--Rectangle') == None:
                restaurant_detail_fields[i.select_one('span.Restaurant__InfoAddress--Rectangle').get_text(strip=True)] \
                = i.select_one('span.Restaurant__InfoAddress--Text').get_text(strip=True)
        parent_text[restaurant.split(".")[0]]=restaurant_detail_fields
#     print(parent_test)    
#         test[category_detail_list] = restaurant_detail_fields
#         detail_page_craw[category_detail_list] = restaurant_detail_fields
#     test = detail_page_craw
    detail_page_craw[category_detail_list] = parent_text

print('\n')


page 없는 데이터 존재 'NoneType' object has no attribute 'select_one'
page 없는 데이터 존재 'NoneType' object has no attribute 'select_one'
page 없는 데이터 존재 'NoneType' object has no attribute 'select_one'
page 없는 데이터 존재 'NoneType' object has no attribute 'select_one'
page 없는 데이터 존재 'NoneType' object has no attribute 'select_one'




# 식당 모델 에 데이터 생성

In [7]:
for catetory_name,restaurant_name in detail_page_craw.items():
#     print( restaurant_factors)
    for key, restaurant_factors in restaurant_name.items():
        
#         try:
#             restaurant_factors['title']
#             restaurant_factors['주소']
#             restaurant_factors['지번']
#             restaurant_factors['가격대']

#         except KeyError:
#             print(file_name, '   못만듬')
#             print('\n')
#             continue
        try :
            restaurant_factors['주차']
        except KeyError:
            restaurant_factors['주차'] = '정보없음'    


        try :
            restaurant_factors['전화번호']
        except KeyError:
            restaurant_factors['전화번호'] = '010-0000-0000'
#         try :
#             restaurant_factors['음식 종류']
#         except KeyError:
#             restaurant_factors['음식 종류']= '정보없음'

        print(restaurant_factors['title'])
        print(restaurant_factors['주소'])
        print(restaurant_factors['지번'])
        print(restaurant_factors['음식 종류'])
        print(restaurant_factors['가격대'])
        print(restaurant_factors['전화번호'])
        print(restaurant_factors['주차'])
        unique= catetory_name+restaurant_factors['title'].replace(" ","") + restaurant_factors['주소'].replace(" ","")
        print(unique)
        try:

            Restaurant.objects.create(
                name_address1_unique = unique,
                name                 = restaurant_factors['title'].replace(" ",""),
                address1             = restaurant_factors['주소'].replace(" ",""),
                address2             = restaurant_factors['지번'].replace(" ",""),
                phone                = restaurant_factors['전화번호'].replace(" ",""),
                restaurant_type      = restaurant_factors['음식 종류'].replace(" ",""),
                price_range          = restaurant_factors['가격대'].replace(" ",""),
                parking              = restaurant_factors['주차'].replace(" ",""),
            )
        except OperationalError:
            print('unique 에러여서 넘어감')
            continue




로향양꼬치
서울특별시 관악구 관악로16길 35 푸르빌아파트지번서울시 관악구 봉천동 1601-10 푸르빌아파트
서울시 관악구 봉천동 1601-10 푸르빌아파트
기타 중식
만원-2만원
02-885-5918
무료주차 가능
서울대맛집베스트30곳로향양꼬치서울특별시관악구관악로16길35푸르빌아파트지번서울시관악구봉천동1601-10푸르빌아파트


NameError: name 'OperationalError' is not defined

# 카테고리 아이디 등록

In [25]:
for catetory_name,restaurant_name in detail_page_craw.items():
    for key, restaurant_factors in restaurant_name.items():

        print(restaurant_factors['images']['image_0'])
        unique= catetory_name+restaurant_factors['title'].replace(" ","") + restaurant_factors['주소'].replace(" ","")
        aa=Restaurant.objects.get(name_address1_unique= unique)
        print(aa.name)
        try:

            RestaurantCategory.objects.create(
                restaurant           = Restaurant.objects.get(name_address1_unique= unique ),
                thumbnail            = restaurant_factors['images']['image_0'],
                category             = catetory_name ,

            )
        except IntegrityError:
            print('unique 에러여서 넘어감')
            continue
        



https://mp-seoul-image-production-s3.mangoplate.com/16647/129405_1431781055471394793952539798?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
로향양꼬치
https://mp-seoul-image-production-s3.mangoplate.com/195829/1168227_1559722994014_20961?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
차이나당
https://mp-seoul-image-production-s3.mangoplate.com/1247474_1547185817525922.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
뉴욕택시디저트
https://mp-seoul-image-production-s3.mangoplate.com/272492/1311083_1563755625054_309595?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
안녕베트남
https://mp-seoul-image-production-s3.mangoplate.com/705256_1575706043849051.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
소풍가는날
https://mp-seoul-image-production-s3.mangoplate.com/881793_1512208063612527.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
텐동요츠야
https://mp-seoul-imag

https://mp-seoul-image-production-s3.mangoplate.com/added_restaurants/442981_1449134191626.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
와플잇업
https://mp-seoul-image-production-s3.mangoplate.com/10670/46118_1433260330751389832?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
땡스네이쳐카페
https://mp-seoul-image-production-s3.mangoplate.com/189804/442542_1449037594333_24165?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
리화인와플
https://mp-seoul-image-production-s3.mangoplate.com/117386_1585904407643994.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
카페호스팅
https://mp-seoul-image-production-s3.mangoplate.com/388403/1247423_1584348556717_73941?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
프레스커피
https://mp-seoul-image-production-s3.mangoplate.com/908198_1583341812074245.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
오목눈이
https://mp-seoul-ima

In [46]:
for catetory_name,restaurant_name in detail_page_craw.items():
    
    for key, restaurant_factors in restaurant_name.items():
        unique= catetory_name+restaurant_factors['title'].replace(" ","") + restaurant_factors['주소'].replace(" ","")

        print(restaurant_factors['지번'].split()[1])
#         print(aa.name)
        try:

            RestaurantLocation.objects.create(
                restaurant           = Restaurant.objects.get(name_address1_unique= unique ),
                location             = restaurant_factors['지번'].split()[1],

            )
        except IntegrityError:
            print('unique 에러여서 넘어감')
            continue
        



# class RestaurantLocation(models.Model):
#     restaurant = models.ForeignKey(Restaurant, on_delete=models.PROTECT, verbose_name='지역_식당')
#     location = models.CharField(max_length=30, blank=True,
#                                 verbose_name='카테고리_식당_섬네일')
#     date_joined = models.DateTimeField(auto_now_add=True, verbose_name='가입날짜')
#     date_update = models.DateTimeField(auto_now=True, verbose_name='수정날짜')

관악구
관악구
관악구
관악구
관악구
관악구
관악구
관악구
관악구
관악구
용산구
용산구
용산구
용산구
용산구
용산구
용산구
용산구
용산구
용산구
원주시
원주시
원주시
원주시
원주시
원주시
원주시
원주시
원주시
원주시
중구
영등포구
강서구
중구
서대문구
강남구
서대문구
마포구
동대문구
마포구
중구
서초구
영등포구
영등포구
강남구
종로구
강남구
고양시
강남구
성동구
용산구
용산구
종로구
마포구
용산구
용산구
남동구
서대문구
서대문구
서대문구
서대문구
서대문구
서대문구
서대문구
서대문구
서대문구
서대문구
중구
중구
서대문구
마포구
서대문구
성동구
송파구
관악구
서대문구
원주시
원주시
금정구
금정구
금정구
금정구
금정구
금정구
금정구
금정구
금정구
강릉시
강릉시
강릉시
강릉시
강릉시
강릉시
강릉시
강릉시
강릉시
강릉시
종로구
종로구
종로구
종로구
종로구
종로구
종로구
종로구
종로구
종로구
마포구
마포구
마포구
마포구
마포구
마포구
마포구
마포구
마포구
마포구


In [57]:
for catetory_name,restaurant_name in detail_page_craw.items():
    
    for key, restaurant_factors in restaurant_name.items():
        unique= catetory_name+restaurant_factors['title'].replace(" ","") + restaurant_factors['주소'].replace(" ","")

#         print(restaurant_factors['images'])
        print('\n')
        for image in restaurant_factors['images']:
            print(restaurant_factors['images'][image])

#         print(aa.name)
            try:

                RestaurantImages.objects.create(
                    restaurant           = Restaurant.objects.get(name_address1_unique= unique ),
                    photo                = restaurant_factors['images'][image],
                )
            except IntegrityError:
                print('unique 에러여서 넘어감')
                continue




# class RestaurantImages(models.Model):
#     restaurant = models.ForeignKey(Restaurant, on_delete=models.PROTECT, verbose_name='이미지_식당')
#     photo = models.ImageField(upload_to="restaurant/images", blank=True)  # 처음 이미지를 업로드하면 media 폴더가 자동으로 생성된다.
#     date_joined = models.DateTimeField(auto_now_add=True, verbose_name='가입날짜')
#     date_update = models.DateTimeField(auto_now=True, verbose_name='수정날짜')



https://mp-seoul-image-production-s3.mangoplate.com/16647/129405_1431781055471394793952539798?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/851287_1584803816679634.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/851287_1584803895745307.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1573028_1583036852983929.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1573028_1583036855201775.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/195829/1168227_1559722994014_20961?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1249119_1

https://mp-seoul-image-production-s3.mangoplate.com/966820_1585630093812891.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/966820_1585630094759914.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/217746/510187_1482762821218_31923?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/217746/244033_1583740241204_156095?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/217746/244033_1583740241204_156094?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/217746/244033_1583740241204_156093?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/217746/

https://mp-seoul-image-production-s3.mangoplate.com/1577178_1585867519764922.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1577178_1585867522206752.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1577178_1585867522834856.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1577178_1585867523324490.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/54992_1501425406754171.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/511866_1585706208177996.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/511866_1585706215592664.jpg?fit

https://mp-seoul-image-production-s3.mangoplate.com/1525340_1581653808215419.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/610330_1581308197549474.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/610330_1581308200113972.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/610330_1581308201555086.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/279765/704481_1489323037542_36170?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/879146_1579240411959281.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/879146_1579240412404894.jpg

https://mp-seoul-image-production-s3.mangoplate.com/196857/554117_1577082159913_71895?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/196857/554117_1577082159913_71896?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/1241/reviews/6256d615a3dc.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1241/1257041_1580837411367_92387?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/973239_1579418601334181.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/973239_1579418773117534.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1241/570339_157

https://mp-seoul-image-production-s3.mangoplate.com/2841/1277036_1583244592479_10650?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/2841/1508168_1582767656849_21810?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/added_restaurants/238971_1449300462321.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/195291/816831_1584717101096_9732?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/195291/816831_1584717101096_9731?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/195291/816831_1584717101096_9734?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplat



https://mp-seoul-image-production-s3.mangoplate.com/908198_1583341812074245.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1378573_1582128704474111.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1378573_1582128705679252.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1378573_1582128705891267.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1378573_1582128706088732.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/added_restaurants/521155_1482125874963231.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1520205_15

https://mp-seoul-image-production-s3.mangoplate.com/1091362_1532451273716640.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/24127/1019829_1525604042629_103496?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/465100_1502281899043032.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/722502_1580217372832791.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/722502_1580217373759661.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/722502_1580217373922488.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/722502_1580217374054984.jp

https://mp-seoul-image-production-s3.mangoplate.com/539629_1585579160494848.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/539629_1585579161717799.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/39945/965277_1585328531176_28529?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/added_restaurants/636184_1479201066665951.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1627395_1584783668589190.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/276038/1633775_1584771753571_28135?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/431

https://mp-seoul-image-production-s3.mangoplate.com/355639/1390544_1585606513125_51559?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/355639/1390544_1585606513126_51545?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/355639/1390544_1585606513126_51546?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/355639/1390544_1585606513126_51548?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80


https://mp-seoul-image-production-s3.mangoplate.com/253733/842104_1558168181027_12126?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com/1592769_1580095242765443.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80
https://mp-seoul-image-production-s3.mangoplate.com